In [1]:
#!/usr/bin/env python

from __future__ import print_function

import collections
import csv
import logging

import os

import SimpleITK as sitk

import radiomics
from radiomics import featureextractor


In [2]:
def main():
    outPath = r''
    inputCSV = os.path.join(outPath, 'f.csv')
    outputFilepath = os.path.join(outPath, 'radiomics_features.csv')
    progress_filename = os.path.join(outPath, 'pyrad_log.txt')
    params = os.path.join(outPath, 'exampleSettings', 'MR.yaml')
    # Configure logging
    rLogger = logging.getLogger('radiomics')

    # Set logging level
    #rLogger.setLevel(logging.INFO)  # Not needed, default log level of logger is INFO
    # Create handler for writing to log file
    handler = logging.FileHandler(filename=progress_filename, mode='w')
    handler.setFormatter(logging.Formatter('%(levelname)s:%(name)s: %(message)s'))
    rLogger.addHandler(handler)

    # Initialize logging for batch log messages
    logger = rLogger.getChild('batch')

    # Set verbosity level for output to stderr (default level = WARNING)
    radiomics.setVerbosity(logging.INFO)
    logger.info('pyradiomics version: %s', radiomics.__version__)
    logger.info('Loading CSV')
    
    flists = []
    try:
        with open(inputCSV, 'r') as inFile:
            cr = csv.DictReader(inFile, lineterminator='\n')
            flists = [row for row in cr]
    except Exception:
        logger.error('CSV READ FAILED', exc_info=True)

    logger.info('Loading Done')
    logger.info('Patients: %d', len(flists))

    if os.path.isfile(params):
        extractor = featureextractor.RadiomicsFeatureExtractor(params)
    else:  # Parameter file not found, use hardcoded settings instead
        settings = {}
        settings['binWidth'] = 16
        settings['resampledPixelSpacing'] = None  # [1,1,1]
        settings['interpolator'] = sitk.sitkBSpline
        settings['enableCExtensions'] = True

        extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
        # extractor.enableInputImages(wavelet= {'level': 2})

    logger.info('Enabled input images types: %s', extractor.enabledImagetypes)
    logger.info('Enabled features: %s', extractor.enabledFeatures)
    logger.info('Current settings: %s', extractor.settings)

    headers = None

    for idx, entry in enumerate(flists, start=1):

        logger.info("(%d/%d) Processing Patient (Image: %s, Mask: %s)", idx, len(flists), entry['Image'], entry['Mask'])

        imageFilepath = entry['Image']
        maskFilepath = entry['Mask']
        label = entry.get('Label', None)

        if str(label).isdigit():
            label = int(label)
        else:
            label = None

        if (imageFilepath is not None) and (maskFilepath is not None):
            featureVector = collections.OrderedDict(entry)
            featureVector['Image'] = os.path.basename(imageFilepath)
            featureVector['Mask'] = os.path.basename(maskFilepath)
            
        try:
            featureVector.update(extractor.execute(imageFilepath, maskFilepath, label))
            
            with open(outputFilepath, 'a') as outputFile:
                writer = csv.writer(outputFile, lineterminator='\n')
                if headers is None:
                    headers = list(featureVector.keys())
                    writer.writerow(headers)

                row = []
                for h in headers:
                    row.append(featureVector.get(h, "N/A"))
                writer.writerow(row)
        except Exception:
            logger.error('FEATURE EXTRACTION FAILED', exc_info=True)

In [3]:
if __name__ == '__main__':
    main()


pyradiomics version: v3.0.1
Loading CSV
Loading Done
Patients: 140
Loading parameter file exampleSettings\exampleMR_3mm.yaml
Enabled input images types: {'Original': {}, 'Wavelet': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Current settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidt

Computing ngtdm
Calculating features for wavelet-LLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
(5/140) Processing Patient (Image: D:\ICC\25\25ADC.nrrd, Mask: D:\ICC\25\25ADC-label.nrrd)
Calculating features with label: 1
Loading image and mask
Applying resampling from spacing [1.4063     1.4063     7.00004339] and size [256 256  24] to spacing [1. 1. 1.] and size [81, 64, 68]
Computing shape
Adding image type "Original" with custom settings: {}
Adding image type "Wavelet" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LLH
Calculating features for wavelet-LLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHL
Calculating features for wavelet-LHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm

Computing Wavelet LHL
Calculating features for wavelet-LHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHH
Calculating features for wavelet-LHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLL
Calculating features for wavelet-HLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLH
Calculating features for wavelet-HLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHL
Calculating features for wavelet-HHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHH
Calculating features for wavelet-HHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Calculating feat

Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Calculating features for wavelet-LLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
(14/140) Processing Patient (Image: D:\ICC\44\44ADC.nrrd, Mask: D:\ICC\44\44ADC-label.nrrd)
Calculating features with label: 1
Loading image and mask
Applying resampling from spacing [1.5625     1.5625     7.19999988] and size [256 184  32] to spacing [1. 1. 1.] and size [45, 52, 55]
Computing shape
Adding image type "Original" with custom settings: {}
Adding image type "Wavelet" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LLH
Calculating features for wavelet-LLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHL
Calculating features for w

Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHL
Calculating features for wavelet-LHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHH
Calculating features for wavelet-LHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLL
Calculating features for wavelet-HLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLH
Calculating features for wavelet-HLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHL
Calculating features for wavelet-HHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHH
Calculating features for wavelet-HHH image
Computing firsto

...

Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLH
Calculating features for wavelet-HLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHL
Calculating features for wavelet-HHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHH
Calculating features for wavelet-HHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Calculating features for wavelet-LLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
(122/140) Processing Patient (Image: D:\ICC\21\21D.nrrd, Mask: D:\ICC\21\21D-label.nrrd)
Calculating features with label: 1
Loading image and mask
Applying resampling from spacing [1.0795455  1.0795455  3.49999595] and size [352 286  64] to spacing [1. 1. 1.] and size [91, 91, 75]
Computing shape
Addin

Computing shape
Adding image type "Original" with custom settings: {}
Adding image type "Wavelet" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LLH
Calculating features for wavelet-LLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHL
Calculating features for wavelet-LHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHH
Calculating features for wavelet-LHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLL
Calculating features for wavelet-HLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLH
Calculating features for wavelet-HLH image
Computing firstor

Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLH
Calculating features for wavelet-HLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHL
Calculating features for wavelet-HHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHH
Calculating features for wavelet-HHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Calculating features for wavelet-LLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
(131/140) Processing Patient (Image: D:\ICC\39\39D.nrrd, Mask: D:\ICC\39\39D-label.nrrd)
Calculating features with label: 1
Loading image and mask
Applying resampling from spacing [0.7422     0.7422     1.99999409] and size [512 512  80] to spacing [1. 1. 1.] and size [102, 86, 70]
Computing shape
Adding image type "O

Computing shape
Adding image type "Original" with custom settings: {}
Adding image type "Wavelet" with custom settings: {}
Calculating features for original image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LLH
Calculating features for wavelet-LLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHL
Calculating features for wavelet-LHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet LHH
Calculating features for wavelet-LHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLL
Calculating features for wavelet-HLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLH
Calculating features for wavelet-HLH image
Computing firstor

Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HLH
Calculating features for wavelet-HLH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHL
Calculating features for wavelet-HHL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Computing Wavelet HHH
Calculating features for wavelet-HHH image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
Calculating features for wavelet-LLL image
Computing firstorder
Computing glcm
Computing glrlm
Computing glszm
Computing gldm
Computing ngtdm
(140/140) Processing Patient (Image: D:\ICC\116\116D.nrrd, Mask: D:\ICC\116\116D-label.nrrd)
Calculating features with label: 1
Loading image and mask
Applying resampling from spacing [0.8203     0.8203     2.80043578] and size [512 512  80] to spacing [1. 1. 1.] and size [65, 53, 42]
Computing shape
Adding image type